#PART 2 Indexing the Factory Contract

##Task 1: Fetch Deployed Contracts
###This task connects to the Ethereum mainnet using the Infura RPC provider and interacts with the DefaultCollateralFactory contract to retrieve all deployed DefaultCollateral contract addresses. The contract's ABI defines the getDeployedContracts function, which is called to fetch the list of addresses. Once retrieved, the addresses are printed to the console and saved to a deployed_contracts.json file in the local directory. This ensures that the contract addresses are stored for further processing in Task 2. Any errors during the process, such as connection issues or incorrect ABI, are caught and logged for debugging.
---



In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv
import json

load_dotenv()
INFURA_API_KEY = os.getenv("INFURA_API_KEY") #REPLACE

#  Connect to Ethereum Mainnet
provider_url = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
web3 = Web3(Web3.HTTPProvider(provider_url))

#  Define Factory Contract Details
factory_address = "0xAEb6bdd95c502390db8f52c8909F703E9Af6a346"  # VaultFactory contract address
factory_abi = [{"inputs":[{"internalType":"address","name":"owner_","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"AlreadyBlacklisted","type":"error"},{"inputs":[],"name":"AlreadyWhitelisted","type":"error"},{"inputs":[],"name":"EntityNotExist","type":"error"},{"inputs":[],"name":"InvalidImplementation","type":"error"},{"inputs":[],"name":"InvalidVersion","type":"error"},{"inputs":[],"name":"NotOwner","type":"error"},{"inputs":[],"name":"OldVersion","type":"error"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"OwnableInvalidOwner","type":"error"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"OwnableUnauthorizedAccount","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"entity","type":"address"}],"name":"AddEntity","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint64","name":"version","type":"uint64"}],"name":"Blacklist","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"entity","type":"address"},{"indexed":false,"internalType":"uint64","name":"newVersion","type":"uint64"}],"name":"Migrate","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Whitelist","type":"event"},{"inputs":[{"internalType":"uint64","name":"version","type":"uint64"}],"name":"blacklist","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint64","name":"version","type":"uint64"}],"name":"blacklisted","outputs":[{"internalType":"bool","name":"value","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint64","name":"version","type":"uint64"},{"internalType":"address","name":"owner_","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"create","outputs":[{"internalType":"address","name":"entity_","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"index","type":"uint256"}],"name":"entity","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint64","name":"version","type":"uint64"}],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"entity_","type":"address"}],"name":"isEntity","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lastVersion","outputs":[{"internalType":"uint64","name":"","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"entity_","type":"address"},{"internalType":"uint64","name":"newVersion","type":"uint64"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"migrate","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"totalEntities","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"implementation_","type":"address"}],"name":"whitelist","outputs":[],"stateMutability":"nonpayable","type":"function"}]

# Initialize the factory contract
factory_contract = web3.eth.contract(address=Web3.to_checksum_address(factory_address), abi=factory_abi)

#  Fetch All Deployed DefaultCollateral Contracts
def get_deployed_contracts():
    try:
        print("Fetching deployed DefaultCollateral contracts...")
        deployed_contracts = factory_contract.functions.getDeployedContracts().call()
        print("Deployed Contracts:", deployed_contracts)

        # Save to a JSON file
        with open("deployed_contracts.json", "w") as f:
            json.dump(deployed_contracts, f, indent=4)
        print("Saved deployed contracts to 'deployed_contracts.json'")
    except Exception as e:
        print(f"Error fetching deployed contracts: {e}")

if __name__ == "__main__":
    get_deployed_contracts()


##Task 2: Fetch Input Tokens

###This task reads the list of deployed DefaultCollateral contract addresses from the deployed_contracts.json file generated in Task 1. For each contract address, the script initializes a contract instance using its ABI and interacts with the inputToken function to retrieve the token address used for deposit and withdraw operations. The retrieved token addresses, along with their corresponding DefaultCollateral contract addresses, are stored in a list and saved to an input_tokens.json file. This provides a comprehensive mapping of all deployed contracts and their associated input tokens. Errors, such as missing data or unexpected function behavior, are logged for troubleshooting.









In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv
import json

load_dotenv()
INFURA_API_KEY = os.getenv("INFURA_API_KEY")

provider_url = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
web3 = Web3(Web3.HTTPProvider(provider_url))

collateral_abi = [
    {
        "constant": True,
        "inputs": [],
        "name": "inputToken",
        "outputs": [{"name": "", "type": "address"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    }
]

def fetch_input_tokens():
    try:
        # Load deployed contracts from JSON file
        with open("deployed_contracts.json", "r") as f:
            deployed_contracts = json.load(f)

        token_data = []
        for address in deployed_contracts:
            try:
                print(f"Fetching input token for DefaultCollateral contract: {address}")
                collateral_contract = web3.eth.contract(
                    address=Web3.to_checksum_address(address), abi=collateral_abi
                )
                input_token_address = collateral_contract.functions.inputToken().call()
                print(f"Input Token Address: {input_token_address}")
                token_data.append({"collateral_contract": address, "input_token": input_token_address})
            except Exception as e:
                print(f"Error fetching input token for {address}: {e}")

        # Save results to a JSON file
        with open("input_tokens.json", "w") as f:
            json.dump(token_data, f, indent=4)
        print("Saved input token data to 'input_tokens.json'")
    except Exception as e:
        print(f"Error processing contracts: {e}")

if __name__ == "__main__":
    fetch_input_tokens()


#Part 3: Implementing Deposit and Withdraw Functions

## TASK 1 IMPLEMENTING DEPOSIT FUNCTION

In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv

load_dotenv()
INFURA_API_KEY = os.getenv("INFURA_API_KEY")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")  # Your wallet's private key
WALLET_ADDRESS = os.getenv("WALLET_ADDRESS")  # Your wallet address

 Connect to Ethereum Mainnet
provider_url = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
web3 = Web3(Web3.HTTPProvider(provider_url))

# ABI for DefaultCollateral contract
collateral_abi = [
    {
        "constant": False,
        "inputs": [{"name": "amount", "type": "uint256"}],
        "name": "deposit",
        "outputs": [{"name": "outputAmount", "type": "uint256"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "conversionRate",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    },
]

def deposit(collateral_address, input_amount):
    try:
        # Initialize the contract
        collateral_contract = web3.eth.contract(
            address=Web3.to_checksum_address(collateral_address), abi=collateral_abi
        )

        conversion_rate = collateral_contract.functions.conversionRate().call()
        print(f"Conversion Rate: {conversion_rate}")

        expected_output = input_amount * conversion_rate
        print(f"Expected Output Amount: {expected_output}")

        nonce = web3.eth.get_transaction_count(WALLET_ADDRESS)
        transaction = collateral_contract.functions.deposit(input_amount).build_transaction({
            "chainId": 1,  # Mainnet chain ID
            "gas": 200000,  # Adjust gas limit as needed
            "gasPrice": web3.to_wei("20", "gwei"),  # Adjust gas price as needed
            "nonce": nonce,
            "from": WALLET_ADDRESS,
        })

        signed_txn = web3.eth.account.sign_transaction(transaction, private_key=PRIVATE_KEY)

        txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        print(f"Transaction sent: {web3.to_hex(txn_hash)}")

        receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
        print(f"Transaction receipt: {receipt}")

        return expected_output
    except Exception as e:
        print(f"Error in deposit function: {e}")
        return None

# Step 3: Test the Deposit Function
if __name__ == "__main__":
    collateral_address = "0xAEb6bdd95c502390db8f52c8909F703E9Af6a346"  # Replace with actual contract address
    input_amount = 100  # Replace with the deposit amount (in smallest unit, e.g., wei)

    output = deposit(collateral_address, input_amount)
    if output:
        print(f"Deposit successful. Output Amount: {output}")
    else:
        print("Deposit failed.")


Explanation of Key Components:
Contract Interaction:

The deposit function is defined in the DefaultCollateral contract. It accepts the deposit amount and returns the output amount.
Conversion Rate:

The conversionRate function fetches the rate for converting the deposit token into the output amount. This rate is multiplied by the input amount to calculate the expected output.
Transaction Preparation:

The script constructs the transaction to call the deposit function.
It sets the necessary parameters like gas limit, gas price, and chain ID (1 for Ethereum mainnet).
Transaction Signing and Sending:

The transaction is signed using the private key of the wallet, ensuring it's valid.
The signed transaction is sent to the Ethereum network, and the receipt is fetched after the transaction is mined.
Output Amount:

The script calculates and displays the output amount based on the conversion rate.

## TASK 2 IMPLEMENTING WITHDRAW FUNCTION

In [ ]:
from web3 import Web3
import os
from dotenv import load_dotenv

load_dotenv()
INFURA_API_KEY = os.getenv("INFURA_API_KEY")
PRIVATE_KEY = os.getenv("PRIVATE_KEY")  # Your wallet's private key
WALLET_ADDRESS = os.getenv("WALLET_ADDRESS")  # Your wallet address

#  Connect to Ethereum Mainnet
provider_url = f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"
web3 = Web3(Web3.HTTPProvider(provider_url))

# ABI for DefaultCollateral contract
collateral_abi = [
    {
        "constant": False,
        "inputs": [{"name": "amount", "type": "uint256"}],
        "name": "withdraw",
        "outputs": [{"name": "outputAmount", "type": "uint256"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "conversionRate",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    },
]

def withdraw(collateral_address, input_amount):
    try:
        collateral_contract = web3.eth.contract(
            address=Web3.to_checksum_address(collateral_address), abi=collateral_abi
        )

        conversion_rate = collateral_contract.functions.conversionRate().call()
        print(f"Conversion Rate: {conversion_rate}")

        expected_output = input_amount / conversion_rate
        print(f"Expected Output Amount: {expected_output}")

        nonce = web3.eth.get_transaction_count(WALLET_ADDRESS)
        transaction = collateral_contract.functions.withdraw(input_amount).build_transaction({
            "chainId": 1,  # Mainnet chain ID
            "gas": 200000,  # Adjust gas limit as needed
            "gasPrice": web3.to_wei("20", "gwei"),  # Adjust gas price as needed
            "nonce": nonce,
            "from": WALLET_ADDRESS,
        })

        signed_txn = web3.eth.account.sign_transaction(transaction, private_key=PRIVATE_KEY)

        txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
        print(f"Transaction sent: {web3.to_hex(txn_hash)}")

        receipt = web3.eth.wait_for_transaction_receipt(txn_hash)
        print(f"Transaction receipt: {receipt}")

        # Return the output amount
        return expected_output
    except Exception as e:
        print(f"Error in withdraw function: {e}")
        return None

if __name__ == "__main__":
    collateral_address = "0xAEb6bdd95c502390db8f52c8909F703E9Af6a346"  # Replace with actual contract address
    input_amount = 50  # Replace with the withdrawal amount (in smallest unit, e.g., wei)

    output = withdraw(collateral_address, input_amount)
    if output:
        print(f"Withdrawal successful. Output Amount: {output}")
    else:
        print("Withdrawal failed.")


Explanation of Key Components:
Contract Interaction:

The withdraw function in the contract is called with the input amount.
The ABI specifies the function's input and output structure.
Conversion Rate:

The conversionRate function is called to calculate the expected output amount. Adjust this if the contract handles conversions differently.
Transaction Preparation:

A transaction is created to call the withdraw function, including parameters like gas limit, gas price, and chain ID.
Transaction Signing and Sending:

The transaction is signed and sent using the wallet's private key.
The script waits for the transaction receipt to confirm the withdrawal was successful.
Output Amount:

The script calculates and displays the expected output amount based on the contract's conversion rate.